In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT
from metrics import precision_length, precision_segment, recall_segment, recall_length, route_error, analyze_spatial_skewing_meters, analyze_spatial_skewing_metersV2, analyze_spatial_skewing_metersV3
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import mapping
import tracers as tr
import numpy as np


In [ ]:
#load network data and graph 
network = Network("../osmnx_example/rome/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

ubodt = UBODT.read_ubodt_csv("../osmnx_example/rome/ubodt.txt")

model = FastMapMatch(network,graph,ubodt)

Nodes 88697 edges 188616
[2025-03-17 14:25:10.608] [info] [network.cpp:72] Read network from file ../osmnx_example/rome/edges.shp
[2025-03-17 14:25:13.346] [info] [network.cpp:172] Number of edges 188616 nodes 88697
[2025-03-17 14:25:13.346] [info] [network.cpp:173] Field index: id 20 source 0 target 1
[2025-03-17 14:25:13.544] [info] [network.cpp:176] Read network done.
[2025-03-17 14:25:13.546] [info] [network_graph.cpp:17] Construct graph from network edges start
[2025-03-17 14:25:13.610] [info] [network_graph.cpp:30] Graph nodes 88697 edges 188616
[2025-03-17 14:25:13.610] [info] [network_graph.cpp:31] Construct graph from network edges end
[2025-03-17 14:25:13.611] [info] [ubodt.cpp:208] Reading UBODT file (CSV format) from ../osmnx_example/rome/ubodt.txt
[2025-03-17 14:25:14.430] [info] [ubodt.cpp:236] Read rows 1000000
[2025-03-17 14:25:15.094] [info] [ubodt.cpp:236] Read rows 2000000
[2025-03-17 14:25:16.024] [info] [ubodt.cpp:236] Read rows 3000000
[2025-03-17 14:25:16.644] [i

: 

In [ ]:


#load taxi data
df = pd.read_csv("../osmnx_example/rome/original_taxi_156.csv", sep=";")


wkt = df.loc[df["id"] == 156, "geom"].values[0]
print("raw wtk: ", wkt)

# configuration parameters
k = 10
radius = 400/100_000
gps_error = 75/100_000

fmm_config = FastMapMatchConfig(k,radius,gps_error,obfuscation=False, reverse_tolerance=10)

result_raw = model.match_wkt(wkt,fmm_config)

gt_wtk = loads(result_raw.mgeom.export_wkt())
print(gt_wtk)

In [ ]:
#perturbated data map matching
df_pert = pd.read_csv("../osmnx_example/rome/pert_taxi_156.csv", sep=";")

wkt_pert = df_pert.loc[df["id"] == 156, "geom"].values[0]

k = 8
# devide by 100 000 to go from meter to degrees
radius = 400/100_000
gps_error = 75/100_000
obfuscation_error = 300/100_000
total_error = np.sqrt(gps_error**2 + obfuscation_error**2)
fmm_config = FastMapMatchConfig(k,radius,total_error,obfuscation=False,reverse_tolerance=10)
result_pert = model.match_wkt(wkt_pert,fmm_config)

pert_mm_wtk = loads(result_pert.mgeom.export_wkt())



In [ ]:
# error metric van https://doi.org/10.1145/1653771.1653818
# Load geometries from WKT
print(wkt)
print(gt_wtk)
print(pert_mm_wtk)
print(result_pert.mgeom.export_wkt())

error_metrics = route_error(gt_wtk, pert_mm_wtk)

# Print results
print(f"Correct route length (d0): {error_metrics['d0']:.2f}")
print(f"Erroneously subtracted (d-): {error_metrics['d_minus']:.2f}")
print(f"Erroneously added (d+): {error_metrics['d_plus']:.2f}")
print(f"Error metric: {error_metrics['error']:.4f}")

skewing_metrics = analyze_spatial_skewing_metersV2(gt_wtk, pert_mm_wtk)


In [ ]:
import geopandas as gpd
from shapely.wkt import loads
from shapely.geometry import LineString, Polygon, Point
from shapely.ops import unary_union
import matplotlib.pyplot as plt
from rtree import index
import folium


def find_segment_intersections(line1, line2):
    """Find all intersections between segments of two lines with angles > 5 degrees"""
    intersections = []
    
    #DEBUG
    print(line1)
    print(line2)

    # Create segments for first line
    segments1 = []
    print("calculating segments 1")
    for i in range(len(line1.coords) - 1):
        seg = LineString([line1.coords[i], line1.coords[i+1]])
        segments1.append((i, seg))
    
    # Create segments for second line and build spatial index
    print("calculating segments 2")
    segments2 = []
    idx = index.Index()
    for i in range(len(line2.coords) - 1):
        seg = LineString([line2.coords[i], line2.coords[i+1]])
        segments2.append((i, seg))
        idx.insert(i, seg.bounds)
    
    # Check for intersections using spatial index
    for i, (idx1, seg1) in enumerate(segments1):
        # Find potential intersections using bounding box
        potential_matches = list(idx.intersection(seg1.bounds))
        
        for j, idx2 in enumerate(potential_matches):
            seg2 = segments2[idx2][1]
            if seg1.intersects(seg2):
                intersection = seg1.intersection(seg2)
                if isinstance(intersection, Point):
                    # Calculate angle between segments
                    # Get direction vectors for both segments
                    vec1 = np.array([line1.coords[idx1+1][0] - line1.coords[idx1][0], 
                                    line1.coords[idx1+1][1] - line1.coords[idx1][1]])
                    vec2 = np.array([line2.coords[idx2+1][0] - line2.coords[idx2][0], 
                                    line2.coords[idx2+1][1] - line2.coords[idx2][1]])
                    
                    # Normalize vectors
                    vec1_norm = vec1 / np.linalg.norm(vec1)
                    vec2_norm = vec2 / np.linalg.norm(vec2)
                    
                    # Calculate dot product and angle
                    dot_product = np.dot(vec1_norm, vec2_norm)
                    # Ensure dot product is in valid range for arccos
                    dot_product = np.clip(dot_product, -1.0, 1.0)
                    angle_rad = np.arccos(abs(dot_product))
                    angle_deg = np.degrees(angle_rad)
                    
                    # Only keep intersections with angle > 5 degrees
                    if angle_deg > 5:
                        # Calculate distances along each line
                        dist1 = line1.project(intersection)
                        dist2 = line2.project(intersection)
                        
                        intersections.append({
                            'point': intersection,
                            'seg1_idx': idx1,
                            'seg2_idx': idx2,
                            'dist1': dist1,
                            'dist2': dist2,
                            'angle': angle_deg  # Adding angle to the output for reference
                        })
    
    return intersections

def split_line_at_points(line, split_points):
    """Split a line at given points and return the new line with additional vertices"""
    if not split_points:
        return line
    
    # Convert to list for manipulation
    coords = list(line.coords)
    
    # Sort split points by distance along the line
    split_points.sort(key=lambda x: x['dist'])
    
    # Convert coords to numpy array for faster distance calculation
    coords_array = np.array(coords)
    
    # Insert points
    added = 0
    for point_info in split_points:
        point = point_info['point']
        seg_idx = point_info['seg_idx']
        
        # Skip if the point is already too close to an existing vertex (optimized calculation)
        point_coords = np.array([point.x, point.y])
        squared_dists = np.sum((coords_array - point_coords)**2, axis=1)
        min_squared_dist = np.min(squared_dists)
        
        # Using squared distance comparison (equivalent to 1e-8 for actual distance)
        if min_squared_dist < 1e-16:  # Squared version of 1e-8
            continue
            
        # Insert at the correct position
        insert_pos = seg_idx + 1 + added
        if insert_pos < len(coords):
            coords.insert(insert_pos, (point.x, point.y))
            added += 1
            
            # Update numpy array for future distance calculations
            coords_array = np.array(coords)
    
    return LineString(coords)

from shapely.geometry import LineString, Polygon, Point
from shapely.ops import unary_union
import numpy as np

def calculate_surface_area(wkt, wkt_pert):
    # Parse WKT strings to Shapely geometries
    line1 = wkt
    line2 = wkt_pert
    
    # Find intersections
    raw_intersections = find_segment_intersections(line1, line2)
    
    if raw_intersections:
        # Prepare split points for both lines in one pass
        split_points1 = []
        split_points2 = []
        
        for intr in raw_intersections:
            split_points1.append({
                'point': intr['point'],
                'seg_idx': intr['seg1_idx'],
                'dist': intr['dist1']
            })
            
            split_points2.append({
                'point': intr['point'],
                'seg_idx': intr['seg2_idx'],
                'dist': intr['dist2']
            })
        
        # Add intersection points to both lines
        line1 = split_line_at_points(line1, split_points1)
        line2 = split_line_at_points(line2, split_points2)
    
    # Get coordinates more efficiently
    coords1 = np.array(line1.coords)
    coords2 = np.array(line2.coords)
    
    # Pre-allocate polygon list with approximate size
    estimated_size = min(len(coords1), len(coords2)) - 1
    polygons = []
    #polygons.reserve = estimated_size  # Reserve space if possible
    total_area = 0
    
    # Create polygons between corresponding points - optimized loop
    for i in range(len(coords1) - 1):
        try:
            # Create quadrilateral vertices directly from numpy arrays when possible
            if i < len(coords2) - 1:
                polygon = Polygon([
                    (coords1[i, 0], coords1[i, 1]), 
                    (coords1[i+1, 0], coords1[i+1, 1]), 
                    (coords2[i+1, 0], coords2[i+1, 1]), 
                    (coords2[i, 0], coords2[i, 1])
                ])
                
                # Only check validity if needed
                if polygon.area > 0:
                    if polygon.is_valid:
                        polygons.append(polygon)
                        total_area += polygon.area
            else:
                break  # Exit early if we've run out of corresponding points
        except Exception as e:
            print(f"Error creating polygon at segment {i}: {e}")
    
    # Try to combine all polygons into one
    combined_polygon = None
    if polygons:
        try:
            combined_polygon = unary_union(polygons)
        except Exception as e:
            print(f"Error combining polygons: {e}")
    
    # Create result dictionary
    result = {
        'area': total_area,
        'polygons': polygons,
        'combined_polygon': combined_polygon,
        'line1': line1,
        'line2': line2,
        'intersections': [intr['point'] for intr in raw_intersections]
    }
    
    return result

import contextily as ctx

def plot_comparison1(result):
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Assume input data is in WGS84 (EPSG:4326) and convert to Web Mercator (EPSG:3857)
    crs = "EPSG:4326"
    
    # Plot the polygons
    if result['combined_polygon']:
        gdf_poly = gpd.GeoDataFrame(geometry=[result['combined_polygon']], crs=crs)
        gdf_poly = gdf_poly.to_crs(epsg=3857)
        gdf_poly.plot(ax=ax, color='blue', alpha=0.5)
    else:
        for poly in result['polygons']:
            gdf = gpd.GeoDataFrame(geometry=[poly], crs=crs)
            gdf = gdf.to_crs(epsg=3857)
            gdf.plot(ax=ax, color='lightblue', alpha=0.5)
    
    # Plot the lines
    gdf_line1 = gpd.GeoDataFrame(geometry=[result['line1']], crs=crs)
    gdf_line1 = gdf_line1.to_crs(epsg=3857)
    gdf_line1.plot(ax=ax, color='blue', linewidth=2, label='Original')
    
    gdf_line2 = gpd.GeoDataFrame(geometry=[result['line2']], crs=crs)
    gdf_line2 = gdf_line2.to_crs(epsg=3857)
    gdf_line2.plot(ax=ax, color='red', linewidth=2, label='Perturbed')
    
    # Plot intersection points
    if result['intersections']:
        gdf_points = gpd.GeoDataFrame(geometry=result['intersections'], crs=crs)
        gdf_points = gdf_points.to_crs(epsg=3857)
        gdf_points.plot(ax=ax, color='green', markersize=40, marker='o', edgecolor='k', label='Intersections')
    
    # Add basemap
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
    
    plt.legend()
    plt.title(f"Surface Area: {result['area']:.6f} square units")
    plt.tight_layout()
    return fig

def plot_comparison(result):
    """
    Creates an interactive map using Folium to visualize the comparison results.

    Args:
        result (dict): A dictionary containing the comparison results, including
                       'line1', 'line2', 'polygons', 'combined_polygon', 'intersections', and 'area'.

    Returns:
        folium.Map: A Folium map object.
    """

    # Determine the center of the map based on the input geometries
    all_geoms = [result['line1'], result['line2']]
    if result['combined_polygon']:
        all_geoms.append(result['combined_polygon'])
    else:
        all_geoms.extend(result['polygons'])
    if result['intersections']:
        all_geoms.extend(result['intersections'])

    all_gdf = gpd.GeoDataFrame(geometry=all_geoms, crs="EPSG:4326")
    bounds = all_gdf.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2

    # Create the Folium map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Plot the polygons
    if result['combined_polygon']:
        folium.GeoJson(
            result['combined_polygon'],
            style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'fillOpacity': 0.5}
        ).add_to(m)
    else:
        for poly in result['polygons']:
            folium.GeoJson(
                poly,
                style_function=lambda x: {'fillColor': 'lightblue', 'color': 'lightblue', 'fillOpacity': 0.5}
            ).add_to(m)

    # Plot the lines
    folium.GeoJson(
        result['line1'],
        style_function=lambda x: {'color': 'blue', 'weight': 2}
    ).add_to(m)

    folium.GeoJson(
        result['line2'],
        style_function=lambda x: {'color': 'red', 'weight': 2}
    ).add_to(m)

    # Plot intersection points
    if result['intersections']:
        for point in result['intersections']:
            folium.CircleMarker(
                location=[point.y, point.x],
                radius=5,
                color='green',
                fill=True,
                fill_color='green',
                fill_opacity=1
            ).add_to(m)

    # Add a popup with the surface area
    folium.Marker(
        location=[center_lat, center_lon],
        popup=f"Surface Area: {result['area']:.6f} square units"
    ).add_to(m)

    return m
result = calculate_surface_area(gt_wtk, pert_mm_wtk)
#result = calculate_surface_area(wkt, wkt_pert)
fig = plot_comparison(result)
fig.save("output.html")
#plt.show()
print(f"Surface area between the lines: {result['area']:.6f} square units")

In [ ]:
import numpy as np
from shapely.geometry import LineString, Polygon, Point
import multiprocessing as mp

def parse_linestring(linestring_text):
    """Parse a LINESTRING text representation into a list of coordinates."""
    # Check if input is already a LineString object
    if isinstance(linestring_text, LineString):
        return list(linestring_text.coords)
    
    # Remove 'LINESTRING' and parentheses, then split by commas
    coords_text = linestring_text.replace('LINESTRING', '').replace('(', '').replace(')', '').strip()
    coords_pairs = coords_text.split(',')
    
    # Parse each coordinate pair
    coords = []
    for pair in coords_pairs:
        x, y = map(float, pair.strip().split())
        coords.append((x, y))
    
    return coords

def find_intersections_with_positions(line1, line2):
    """
    Find all intersection points between two LineString objects 
    and their positions along each line.
    """
    shapely_line1 = LineString(line1)
    shapely_line2 = LineString(line2)
    
    # Get intersection points
    intersection = shapely_line1.intersection(shapely_line2)
    print("find intersection fu" , intersection)
    # Process intersection results
    intersection_points = []
    if intersection.is_empty:
        return []
    elif intersection.geom_type == 'Point':
        intersection_points = [(intersection.x, intersection.y)]
    elif intersection.geom_type == 'MultiPoint':
        intersection_points = [(p.x, p.y) for p in intersection.geoms]
    elif intersection.geom_type == 'LineString':
        # If lines overlap, consider the endpoints of the overlap
        intersection_points = [(intersection.coords[0][0], intersection.coords[0][1]), 
                              (intersection.coords[-1][0], intersection.coords[-1][1])]
    elif intersection.geom_type == 'MultiLineString':
        # For multiple overlapping segments
        for line in intersection.geoms:
            intersection_points.extend([(line.coords[0][0], line.coords[0][1]), 
                                       (line.coords[-1][0], line.coords[-1][1])])
    else:
        return []
    
    # Find the exact segment location for each intersection point
    result = []
    for point in intersection_points:
        point_obj = Point(point)
        
        # Find position along line1
        distance1 = shapely_line1.project(point_obj)
        segment1_idx = 0
        cumulative_length = 0
        for i in range(len(line1) - 1):
            segment = LineString([line1[i], line1[i+1]])
            segment_length = segment.length
            if cumulative_length <= distance1 < cumulative_length + segment_length:
                segment1_idx = i
                break
            cumulative_length += segment_length
        
        # Find position along line2
        distance2 = shapely_line2.project(point_obj)
        segment2_idx = 0
        cumulative_length = 0
        for i in range(len(line2) - 1):
            segment = LineString([line2[i], line2[i+1]])
            segment_length = segment.length
            if cumulative_length <= distance2 < cumulative_length + segment_length:
                segment2_idx = i
                break
            cumulative_length += segment_length
        
        result.append((point, segment1_idx, segment2_idx, distance1, distance2))
    
    # Sort by distance along line1 to maintain order
    result.sort(key=lambda x: x[3])
    return result

def extract_subpath(line, start_idx, start_point, end_idx, end_point):
    """Extract a subpath from a line between two points."""
    subpath = []
    
    # Add the start point
    subpath.append(start_point)
    
    # Add intermediate points
    for i in range(start_idx + 1, end_idx + 1):
        subpath.append(line[i])
    
    # Add the end point if it's not the same as the last added point
    if end_point != subpath[-1]:
        subpath.append(end_point)
    
    return subpath

def calculate_polygon_area(path1, path2):
    """Calculate area of polygon formed by two paths."""
    # Create a polygon by joining the two paths
    polygon_points = list(path1) + list(reversed(path2))
    
    # Create a shapely polygon and calculate its area
    try:
        polygon = Polygon(polygon_points)
        if polygon.is_valid:
            return polygon.area
        else:
            # Try to fix invalid polygon
            polygon = polygon.buffer(0)
            return polygon.area if polygon.is_valid else 0
    except Exception as e:
        print(f"Error creating polygon: {e}")
        return 0

def process_segment_pair(args):
    """Process a pair of segments to calculate polygon area - for parallel processing."""
    path1, path2 = args
    return calculate_polygon_area(path1, path2)

def calculate_area_between_traces(trace1_input, trace2_input, num_processes=None):
    """Calculate the total area between two location traces."""
    # Parse input to get coordinates
    trace1 = parse_linestring(trace1_input)
    trace2 = parse_linestring(trace2_input)
    
    # Find all intersection points with their positions
    intersections = find_intersections_with_positions(trace1, trace2)
    print(intersections)
    if len(intersections) < 2:
        return 0.0  # Not enough intersections to form a polygon
    
    # Extract subpaths between consecutive intersections
    subpaths = []
    for i in range(len(intersections) - 1):
        # Extract current pair of intersection information
        p1, segment1_idx1, segment2_idx1, _, _ = intersections[i]
        p2, segment1_idx2, segment2_idx2, _, _ = intersections[i+1]
        
        # Extract subpaths from both traces
        path1 = extract_subpath(trace1, segment1_idx1, p1, segment1_idx2, p2)
        path2 = extract_subpath(trace2, segment2_idx1, p1, segment2_idx2, p2)
        
        subpaths.append((path1, path2))
    
    # Calculate area for each pair of subpaths
    if not num_processes:
        num_processes = mp.cpu_count()
    
    # Use parallel processing to calculate areas
    if len(subpaths) > 1 and num_processes > 1:
        with mp.Pool(processes=num_processes) as pool:
            areas = pool.map(process_segment_pair, subpaths)
    else:
        areas = [calculate_polygon_area(p1, p2) for p1, p2 in subpaths]
    
    # Sum all areas
    total_area = sum(areas)
    
    return total_area

def main():
    # Example usage
    trace1 = "LINESTRING (0 0, 1 1, 2 0, 3 1, 4 0)"
    trace2 = "LINESTRING (0 1, 1 0, 2 1, 3 0, 4 1)"
    
    area = calculate_area_between_traces(gt_wtk, pert_mm_wtk)
    print(f"Area between traces: {area}")

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import multiprocessing as mp

def parse_linestring(linestring_text):
    """Parse a LINESTRING text representation into a list of coordinates."""
    if hasattr(linestring_text, 'coords'):  # Check if it's a LineString object
        return list(linestring_text.coords)
    
    # Remove 'LINESTRING' and parentheses, then split by commas
    coords_text = linestring_text.replace('LINESTRING', '').replace('(', '').replace(')', '').strip()
    coords_pairs = coords_text.split(',')
    
    # Parse each coordinate pair
    coords = []
    for pair in coords_pairs:
        x, y = map(float, pair.strip().split())
        coords.append((x, y))
    
    return coords

def line_segment_intersection(p1, p2, p3, p4):
    """
    Find the intersection point of two line segments.
    p1, p2 define the first line segment, and p3, p4 define the second line segment.
    Returns the intersection point or None if no intersection.
    """
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    x4, y4 = p4
    
    # Calculate the denominator of the intersection formulas
    denominator = (y4 - y3) * (x2 - x1) - (x4 - x3) * (y2 - y1)
    
    # If denominator is 0, lines are parallel or coincident
    if abs(denominator) < 1e-10:
        # Check if lines are coincident
        if abs((x1 - x3) * (y2 - y3) - (y1 - y3) * (x2 - x3)) < 1e-10:
            # Lines are coincident, find overlap
            
            # Convert to parametric form
            # First line: P1 + t(P2 - P1), t in [0, 1]
            # Second line: P3 + s(P4 - P3), s in [0, 1]
            
            # Project points onto a common line
            # For simplicity, we'll use the first line: P1 + t(P2 - P1)
            def project_point(point):
                # Calculate t value for the point on the first line
                dx = x2 - x1
                dy = y2 - y1
                
                if abs(dx) > abs(dy):
                    t = (point[0] - x1) / dx if dx != 0 else 0
                else:
                    t = (point[1] - y1) / dy if dy != 0 else 0
                return t
            
            t3 = project_point((x3, y3))
            t4 = project_point((x4, y4))
            
            # Ensure t3 <= t4
            if t3 > t4:
                t3, t4 = t4, t3
            
            # Check if there's an overlap
            if max(0, t3) <= min(1, t4):  # Overlap exists
                t_min = max(0, t3)
                t_max = min(1, t4)
                
                # Calculate overlap endpoints
                p_min = (x1 + t_min * (x2 - x1), y1 + t_min * (y2 - y1))
                p_max = (x1 + t_max * (x2 - x1), y1 + t_max * (y2 - y1))
                
                # Return both endpoints as a tuple
                return (p_min, p_max)
            
            return None
        
        return None
    
    # Calculate ua and ub
    ua = ((x4 - x3) * (y1 - y3) - (y4 - y3) * (x1 - x3)) / denominator
    ub = ((x2 - x1) * (y1 - y3) - (y2 - y1) * (x1 - x3)) / denominator
    
    # Check if the intersection is within both line segments
    if 0 <= ua <= 1 and 0 <= ub <= 1:
        # Calculate the intersection point
        x = x1 + ua * (x2 - x1)
        y = y1 + ua * (y2 - y1)
        return (x, y)
    
    return None

def find_intersections_with_positions(trace1, trace2):
    """Find all intersections between two traces and their positions."""
    intersections = []
    
    # Check each pair of line segments
    for i in range(len(trace1) - 1):
        for j in range(len(trace2) - 1):
            p1, p2 = trace1[i], trace1[i+1]
            p3, p4 = trace2[j], trace2[j+1]
            
            intersection = line_segment_intersection(p1, p2, p3, p4)
            
            if intersection:
                # If we got a tuple, it's a coincident segment
                if isinstance(intersection, tuple):
                    # Add both endpoints of the overlap
                    start_point, end_point = intersection
                    
                    # Calculate distances along each trace
                    distance1_start = calculate_distance_along_trace(trace1, i, start_point)
                    distance2_start = calculate_distance_along_trace(trace2, j, start_point)
                    
                    distance1_end = calculate_distance_along_trace(trace1, i, end_point)
                    distance2_end = calculate_distance_along_trace(trace2, j, end_point)
                    
                    intersections.append((start_point, i, j, distance1_start, distance2_start))
                    intersections.append((end_point, i, j, distance1_end, distance2_end))
                else:
                    # Single intersection point
                    # Calculate distances along each trace
                    distance1 = calculate_distance_along_trace(trace1, i, intersection)
                    distance2 = calculate_distance_along_trace(trace2, j, intersection)
                    
                    intersections.append((intersection, i, j, distance1, distance2))
    
    # Sort intersections by their distance along trace1
    intersections.sort(key=lambda x: x[3])
    
    return intersections

def calculate_distance_along_trace(trace, segment_idx, point):
    """Calculate the distance along a trace to a specific point."""
    distance = 0
    
    # Add distances of complete segments
    for i in range(segment_idx):
        p1, p2 = trace[i], trace[i+1]
        distance += np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
    
    # Add distance from the start of the current segment to the point
    p1 = trace[segment_idx]
    # Ensure point is a tuple of coordinates
    if isinstance(point, tuple) and len(point) == 2:
        distance += np.sqrt((point[0] - p1[0])**2 + (point[1] - p1[1])**2)
    
    return distance

def extract_subpath(trace, start_idx, start_point, end_idx, end_point):
    """Extract a subpath from a trace between two points."""
    subpath = []
    
    # Add the start point
    subpath.append(start_point)
    
    # Add intermediate points
    current_idx = start_idx + 1
    while current_idx <= end_idx and current_idx < len(trace):
        subpath.append(trace[current_idx])
        current_idx += 1
    
    # Add the end point if it's not the same as the last added point
    if not subpath or end_point != subpath[-1]:
        subpath.append(end_point)
    
    return subpath

def is_valid_point(point):
    """Check if a point is a valid coordinate pair."""
    return (isinstance(point, tuple) and len(point) == 2 and
            isinstance(point[0], (int, float)) and 
            isinstance(point[1], (int, float)))

def calculate_polygon_area(polygon_points):
    """Calculate the area of a polygon using the Shoelace formula."""
    # Ensure we have at least 3 points to form a polygon
    if len(polygon_points) < 3:
        return 0.0
    
    # Filter out invalid points
    valid_points = [p for p in polygon_points if is_valid_point(p)]
    
    if len(valid_points) < 3:
        return 0.0
    
    # Ensure the polygon is closed
    if valid_points[0] != valid_points[-1]:
        valid_points.append(valid_points[0])
    
    n = len(valid_points)
    area = 0.0
    
    for i in range(n - 1):
        area += valid_points[i][0] * valid_points[i+1][1]
        area -= valid_points[i+1][0] * valid_points[i][1]
    
    area = abs(area) / 2.0
    return area

def process_segment_pair(args):
    """Process a pair of segments to calculate polygon area - for parallel processing."""
    try:
        path1, path2 = args
        if not path1 or not path2:
            return 0.0
        
        # Create a closed polygon by connecting the two paths
        polygon_points = list(path1) + list(reversed(path2))
        
        # Check if polygon_points contains any invalid points
        for point in polygon_points:
            if not is_valid_point(point):
                # If we found an invalid point, filter all points
                return calculate_polygon_area([p for p in polygon_points if is_valid_point(p)])
        
        return calculate_polygon_area(polygon_points)
    except Exception as e:
        print(f"Error in process_segment_pair: {e}")
        return 0.0

def calculate_area_between_traces(trace1_input, trace2_input, num_processes=None):
    """Calculate the total area between two location traces."""
    # Parse input to get coordinates
    trace1 = parse_linestring(trace1_input)
    trace2 = parse_linestring(trace2_input)
    
    # Find all intersection points with their positions
    intersections = find_intersections_with_positions(trace1, trace2)
    
    if len(intersections) < 2:
        return 0.0  # Not enough intersections to form a polygon
    
    # Extract subpaths between consecutive intersections
    subpaths = []
    for i in range(len(intersections) - 1):
        # Extract current pair of intersection information
        p1, segment1_idx1, segment2_idx1, _, _ = intersections[i]
        p2, segment1_idx2, segment2_idx2, _, _ = intersections[i+1]
        
        # Extract subpaths from both traces
        path1 = extract_subpath(trace1, segment1_idx1, p1, segment1_idx2, p2)
        path2 = extract_subpath(trace2, segment2_idx1, p1, segment2_idx2, p2)
        
        subpaths.append((path1, path2))
    
    # Calculate area for each pair of subpaths
    if not num_processes:
        num_processes = mp.cpu_count()
    
    # Use parallel processing to calculate areas
    try:
        if len(subpaths) > 1 and num_processes > 1:
            with mp.Pool(processes=num_processes) as pool:
                areas = pool.map(process_segment_pair, subpaths)
        else:
            areas = [process_segment_pair(subpath) for subpath in subpaths]
    except Exception as e:
        print(f"Error in parallel processing: {e}")
        # Fallback to sequential processing
        areas = []
        for subpath in subpaths:
            try:
                areas.append(process_segment_pair(subpath))
            except Exception as e:
                print(f"Error processing subpath: {e}")
                areas.append(0.0)
    
    # Sum all areas
    total_area = sum(areas)
    
    return total_area

def main():
    # Example usage
    trace1 = "LINESTRING (0 3, 1 1, 2 10, 3 1, 4 0)"
    trace2 = "LINESTRING (0 1, 1 0, 2 1, 3 0, 4 1)"
    
    area = calculate_area_between_traces(trace1, trace2)
    print(f"Area between traces: {area}")

if __name__ == "__main__":
    main()